## CIFAR-10 UAP - Attack Generation
We generate (via SGD), visualize, and evaluate the following UAPs:
1. Untargeted UAP
2. Targeted UAP

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import torch

sys.path.append(os.path.realpath('..'))

from attacks import uap_sgd
from utils import  evaluate

dir_uap = '../uaps/cifar10/'

# load model and data

In [2]:
from robustbench.utils import load_model, clean_accuracy
model = load_model(model_name='Carmon2019Unlabeled', dataset='cifar10', threat_model='Linf')

In [3]:
from robustbench.data import load_cifar10
import json

x_test, y_test = load_cifar10()

with open('config_ids_source_targets.json','r') as f:
    config = json.load(f)
    
ids = config['ids']
x_test_correct , y_test_correct = x_test[ids,:,:,:] , y_test[ids]

Files already downloaded and verified


In [4]:
if torch.cuda.is_available():
    print('using cuda')
    model.cuda()
    x_test_correct = x_test_correct.cuda()
    y_test_correct = y_test_correct.cuda()
    print(x_test_correct.is_cuda, y_test_correct.is_cuda, next(model.parameters()).is_cuda)

using cuda
True True True


In [5]:
from torch.utils.data import Dataset, DataLoader


class MyDataset(Dataset):
    def __init__(self, inputs, labels):
        self.inputs = inputs
        self.labels = labels

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        input_example = self.inputs[idx]
        label_example = self.labels[idx]
        return input_example, label_example

my_dataset = MyDataset(x_test_correct, y_test_correct)

batch_size = 10
#num_workers = 4
test_loader = DataLoader(my_dataset, batch_size=batch_size, shuffle=True)

In [6]:
#accuracy should be equal to 1
test_acc = clean_accuracy(model, x_test_correct, y_test_correct)
print('Test accuracy', test_acc)

Test accuracy 1.0


## 1. Untargeted UAP
Recommended to try several restarts and choose the best untargeted UAP

In [7]:
nb_epoch = 20
eps = 10 / 255
beta = 10
uap, losses = uap_sgd(model, test_loader, nb_epoch, eps, beta)

epoch 1/20
batch_delta.grad is None at iteration  0
epoch 2/20


KeyboardInterrupt: 

In [ ]:
# visualize UAP
plt.imshow(np.transpose(((uap / eps) + 1) / 2, (1, 2, 0)))

In [ ]:
losses_nograd = []
for loss in losses:
    losses_nograd.append(loss.detach().numpy()) 

In [ ]:
# plot loss
plt.plot(losses_nograd)

In [ ]:
# evaluate
_, _, _, _, outputs, labels = evaluate(model, test_loader, uap = uap)
print('Robust ccuracy:', sum(outputs == labels) / len(labels))

In [ ]:
# plot histogram
plt.title('Clean test set distribution')
plt.hist(labels)

In [ ]:
# plot histogram
plt.title('UAP test set distribution')
plt.hist(outputs)

## 2. Targeted UAP

In [ ]:
nb_epoch = 10
eps = 10 / 255
y_target = 7
beta = 10
uap, losses = uap_sgd(model, testloader, nb_epoch, eps, beta, y_target = y_target)

In [ ]:
# visualize UAP
plt.imshow(np.transpose(((uap / eps) + 1) / 2, (1, 2, 0)))

In [ ]:
# plot loss
plt.plot(losses)

In [ ]:
# evaluate
_, _, _, _, outputs, labels = evaluate(model, testloader, uap = uap)
print('Accuracy:', sum(outputs == labels) / len(labels))
print('Targeted success rate:', sum(outputs == y_target) / len(labels))

In [ ]:
# plot histogram
plt.title('Clean test set distribution')
plt.hist(labels)

In [ ]:
# plot histogram
plt.title('UAP test set distribution')
plt.hist(outputs)